In [ ]:
# %%bash
# pip install --upgrade pip
# pip install haystack

In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4

In [ ]:
!pip install farm-haystack[colab]
!pip install 'farm-haystack[elasticsearch]'

In [ ]:
!pip install farm-haystack

In [ ]:
!pip install farm-haystack[inference]

logging and drive

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


elastic search

In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 5

In [ ]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


json and training dataset

In [ ]:
import json

In [ ]:
# Read the contents of the JSON file
with open("/content/drive/MyDrive/Federal_Laws.json", 'r', encoding='utf-8-sig') as f:
    file_contents = f.read()

train_data = json.loads(file_contents)

In [ ]:
train_data

Reader and document_store_TRAIN

In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
from haystack.document_stores.memory import InMemoryDocumentStore
document_store_TRAIN = InMemoryDocumentStore()

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
# Convert the JSON data into Haystack format and add it to the document store
for item in train_data["data"]:
    for paragraph in item["paragraphs"]:
        context = paragraph["context"]
        document_store_TRAIN.write_documents([{"content": context, "meta": {"title": "Federal Laws"}}])

Train reader

In [ ]:
# Train the reader
reader.train(data_dir="/content/drive/MyDrive", train_filename="Federal_Laws.json", n_epochs=10, use_gpu=True)
# reader.train_from_documents(document_store=document_storeExtra, n_epochs=2)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.data_handler.data_silo:
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|
 (o)(o)------'\ _ /     ( )
 
INFO:haystack.modeling.data_handler.data_silo:LOADING TRAIN DATA
INFO:haystack.modeling.data_handler.data_silo:==================
INFO:haystack.modeling.data_handler.data_silo:Loading train set from: /content/drive/MyDrive/Federal_Laws.json 
Preprocessing dataset: 100%|██████████| 1/1 [00:00<00:00,  4.79 Dicts/s]
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:LOADING DEV DATA
INFO:haystack.modeling.data_handler.data_silo:=================
INFO:haystack.modeling.data_handler.data_silo:No dev set is being loaded
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:LOADING TEST DATA
INFO:haystack.modeling.data_handler.data_s

In [ ]:
# Save the trained Reader model
reader.save(directory="/content/drive/MyDrive/reader_model")

INFO:haystack.nodes.reader.farm:Saving reader model to /content/drive/MyDrive/reader_model


In [ ]:
# from haystack.utils.early_stopping import EarlyStopping

# early_stopping = EarlyStopping(
#     metric='top_n_accuracy',
#     save_dir='early_stop_model',
#     mode='max',
#     patience=10,
#     min_delta=0.001,
#     min_evals=0,
# )
# reader = FARMReader(model_name_or_path='deepset/roberta-base-squad2', use_gpu=True)
# reader.train(
#     data_dir='/content/drive/MyDrive',
#     train_filename='Health_Law.json',
#     use_gpu=True,
#     early_stopping=early_stopping
# )

BM25 document_store and retriever

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

document_store_BM25 = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store_BM25)

In [ ]:
# !pip install 'farm-haystack[faiss]'

In [ ]:
# from haystack.document_stores import FAISSDocumentStore
# # from haystack import TfidfRetriever

# # Initialize the document store
# document_store_FAISS = FAISSDocumentStore()

Second Dataset write to BM25 document

In [ ]:
from haystack import Document

# Step 2: Read the text file
file_path = "/content/drive/MyDrive/Federal_Laws_All.txt"
with open(file_path, "r") as file:
    text = file.read()

# Step 3: Create a Document object
document = Document(content=text, meta={"file_path": file_path})

# Step 4: Write the document to the document store
document_store_BM25.write_documents([document])

Updating BM25 representation...: 100%|██████████| 1/1 [00:00<00:00, 83.88 docs/s]


Pipeline

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

# Initialize the ExtractiveQAPipeline with the trained Reader and Elasticsearch Document Store
retriever = BM25Retriever(document_store=document_store_BM25)
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
question = input("What is your question? ")

What is your question? What amendmt gave women the right to vte?


In [ ]:
prediction = pipe.run(query=question, debug=True)

Inferencing Samples: 100%|██████████| 4/4 [00:02<00:00,  1.36 Batches/s]


In [ ]:
[a.answer for a in prediction['answers']][0]

'The Nineteenth Amendment of the United States Constitution'

In [ ]:
from pprint import pprint

pprint(prediction)

In [ ]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="all" ## Choose from `minimum`, `medium`, and `all`
)